In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Import CIFAR-100 and Resize

In [ ]:
resize = transforms.Compose([transforms.Resize(64), transforms.ToTensor()])

In [ ]:
train_set = datasets.CIFAR100(root="data", train=True, download=True, transform=resize)

Files already downloaded and verified


In [ ]:
test_set = datasets.CIFAR100(root="data", train=False, download=True, transform=resize)

Files already downloaded and verified


In [ ]:
train_set[0][0].shape

torch.Size([3, 64, 64])

In [ ]:
test_set[0][0].shape

torch.Size([3, 64, 64])

# VGG16

In [ ]:
import torch.nn as nn
import torchvision.models as models


model = models.vgg16(pretrained=True) #130million+ parameters

#Freeze all model parameters
for param in model.parameters():
    param.requires_grad = False


#Add on classifier
# Add on classifier
n_classes=100
n_inputs=4096
model.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, n_classes))




In [ ]:
model=model.to(device)

In [ ]:
device

device(type='cuda')

In [ ]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Training

In [ ]:
train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=64, shuffle=True)

In [ ]:
criterion=torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [ ]:
n_epochs=10
train_loss=[]
for epoch in range(n_epochs):
  for data, targets in train_dataloader:
    data=data.to(device)
    targets=targets.to(device)
    # Generate predictions
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    train_loss.append(loss)
    #Reset the gradients
    optimizer.zero_grad()
    # Backpropagation
    loss.backward()
    # Update model parameters
    optimizer.step()
  #Print loss
  print(loss)  

tensor(3.2249, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.1547, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.1565, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.9786, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.4669, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.0978, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2238, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.4265, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.0060, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.9788, device='cuda:0', grad_fn=<NllLossBackward>)


# Testing

In [ ]:
test_acc=0
for data, targets in test_dataloader:
  data=data.to(device)
  targets=targets.to(device)
  log_ps = model(data)
  # Convert to probabilities
  ps = torch.exp(log_ps)
  #Get index of class label
  _,preds=torch.max(ps,1)
  #Get accuracy
  test_acc += torch.sum(preds == targets)


In [ ]:
test_acc/len(test_set)

tensor(0.3078, device='cuda:0')

In [ ]:
train_acc=0
for data, targets in train_dataloader:
  data=data.to(device)
  targets=targets.to(device)
  log_ps = model(data)
  # Convert to probabilities
  ps = torch.exp(log_ps)
  #Get index of class label
  _,preds=torch.max(ps,1)
  #Get accuracy
  train_acc += torch.sum(preds == targets)


In [ ]:
train_acc/len(train_set)

tensor(0.4064, device='cuda:0')

In [ ]:
train_acc

tensor(20321, device='cuda:0')